<a href="https://colab.research.google.com/github/jerichosy/THS-STX_user-idea-similarity-for-fixation/blob/main/Similarity_Metric_on_ChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

for context these are the user ideas the users throughout the session.

we want to know how the ideas evolved/progressed throughout its iteration. that is, what is the similarity score of the ideas to each other as it iterated using ChatGPT? A higher similarity score between the iteration of ideas may indicate the users remain fixated on the idea and didn't explore/branch out.

In [ ]:
# Install the required libraries
!pip install boto3 cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00


In [ ]:
import cohere
import numpy as np
from scipy.spatial.distance import cosine
import os
import boto3
import json

# Initialize Cohere client
co = cohere.Client('your_api_key_here')

# Set AWS credentials and region
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

# Initialize the Bedrock client
bedrock = boto3.client(service_name='bedrock-runtime')
model_id = "cohere.embed-english-v3"

def get_embeddings(texts):
    embeddings = []
    for text in texts:
        body = json.dumps({
            "texts": [text],
            "input_type": "clustering"
        })
        response = bedrock.invoke_model(
            body=body,
            modelId=model_id,
            accept="application/json",
            contentType="application/json"
        )
        response_body = json.loads(response['body'].read())
        embeddings.append(response_body['embeddings'][0])
    return embeddings

def calculate_similarity_scores(data):
    results = {}

    for user, ideas in data.items():
        all_similarities = []
        print(f"-- Analyzing {user} --")  # DEBUG info

        if len(ideas) > 1:
            for i in range(len(ideas) - 1):
                pair = (ideas[i], ideas[i + 1])
                similarity = process_pair(pair)
                all_similarities.append(similarity)

        if all_similarities:
            mean_similarity = np.mean(all_similarities)
            std_similarity = np.std(all_similarities)

            results[user] = {
                'mean_similarity': mean_similarity,
                'std_similarity': std_similarity,
                'similarity_scores': all_similarities
            }
        else:
            results[user] = {
                'mean_similarity': None,
                'std_similarity': None,
                'similarity_scores': None
            }

    return results

def process_pair(pair):
    embeddings = get_embeddings(pair)
    similarity = 1 - cosine(embeddings[0], embeddings[1])
    print(f"Pair: {pair}, Similarity: {similarity:.4f}")  # DEBUG info
    return similarity

# New dataset schema
data = {
    "user 2": [
        "Optimal locations, food types, and food prices for attracting college students to eat at my restaurant.",
        "Further developing the restaurant to expand, keep, and grow the customer base, in terms of the optimal locations, food types, and food prices."
    ],
    "user 5": [
        "Features for a social media platform that doesn't distract users and promote productivity."
    ],
    "user 7": [
        "Unsure of new trends for putting up a restaurant where college students can dine in.",
        "Combating inevitable increase of prices because of inflation while still securing customers long-term.",
        "Is using only social media to promote the new restaurant a good option?",
        "Loyalty programs for encouraging students to dine in more at my potential restaurant business.",
        "What is one thing I can do to attract customers without going all-out with my promotions?"
    ],
    "user 8": [
        "List of suggested features for a social media aimed at encouraging professional growth.",
        "How can we implement Project Showcases and Job Board Integration?"
    ],
    "user 9": [
        "Is gamification of job posting and the like a good idea?",
        "Other aspects to gamify?",
        "Event hosting for job opportunities."
    ],
    "user 11": [
        "Restaurant aimed at a younger demographic that will remain loyal even after they grow older.",
        "Factors to consider and weigh in in creating a brand identity.",
        "Other ways to promote aside from seasonal menus since options are limited in an equatorial country.",
        "How to create an impactful and unique brand identity despite being a startup without much funding?"
    ]
}

# Calculate similarity scores
similarity_results = calculate_similarity_scores(data)

# Print results
all_mean_similarities = []
for user, result in similarity_results.items():

    print(f"{user}:")
    if result['mean_similarity'] is not None:
        print(f"  Mean similarity: {result['mean_similarity']:.4f} (std: {result['std_similarity']:.4f})")
        print(f"  Similarity scores: {result['similarity_scores']}")

        all_mean_similarities.append(result['mean_similarity'])
    else:
        print("  Not enough ideas to calculate similarity.")
    print()

print(all_mean_similarities)
print(f"{np.mean(all_mean_similarities)=}")

-- Analyzing user 2 --
Pair: ('Optimal locations, food types, and food prices for attracting college students to eat at my restaurant.', 'Further developing the restaurant to expand, keep, and grow the customer base, in terms of the optimal locations, food types, and food prices.'), Similarity: 0.6738
-- Analyzing user 5 --
-- Analyzing user 7 --
Pair: ('Unsure of new trends for putting up a restaurant where college students can dine in.', 'Combating inevitable increase of prices because of inflation while still securing customers long-term.'), Similarity: 0.2917
Pair: ('Combating inevitable increase of prices because of inflation while still securing customers long-term.', 'Is using only social media to promote the new restaurant a good option?'), Similarity: 0.2791
Pair: ('Is using only social media to promote the new restaurant a good option?', 'Loyalty programs for encouraging students to dine in more at my potential restaurant business.'), Similarity: 0.4564
Pair: ('Loyalty progra

### Analysis Methodology



### Results and Discussion

#### User Analysis



### Interpretation Summary


### Conclusions


### Limitations


### Scratch code